In [1]:
import re
from Bio import AlignIO

In [2]:
gene_name_list = []
with open('../data/roary_result/core_alignment_header.embl', 'r') as file:
    for item in file:
        item = item.rstrip()
        if '/locus_tag' in item:
            gene_name = re.split('locus_tag=', item)[1]
            gene_name_list.append(gene_name)

In [3]:
out_gene_start_list = []
out_gene_end_list = []
out_alignment = None
alignment_length_list = []
current_length = 0
for gene_name in gene_name_list:
    aln_dir = "/share/data_2/lxy/alto/project/brucella_wgs_Yunnan_2024/pipelines/denovo_ssembly/data/split_gene_aln/trim_split_aln"
    alignment = AlignIO.read(aln_dir + "/" + gene_name + "-trim.aln", "fasta")
    alignment_length = alignment.get_alignment_length()
    if out_alignment is not None:
        out_alignment += alignment
        out_gene_start_list.append(current_length + 1)
    else:
        out_alignment = alignment
        out_gene_start_list.append(1)
    out_gene_end_list.append(current_length + alignment_length)
    current_length += alignment_length
    alignment_length_list.append(alignment_length)

In [4]:
out_range_list = ['-'.join([str(s), str(e)]) for s, e in zip(out_gene_start_list, out_gene_end_list)]
with open('../data/trimal_result/core_alignment_header_trim.part', 'w') as f:    
    for gene_name, gene_range in zip(gene_name_list, out_range_list):
        f.write('DNA, ' + gene_name + ' = ' + gene_range + '\n')

In [5]:
AlignIO.write(out_alignment, "../data/trimal_result/core_gene_alignment_trim.aln", "fasta")

1